In [3]:
%conda install -c bioconda emboss


Solving environment: / 
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2020.4.5.1-py37_0, anaconda/osx-64::conda-4.8.3-py37_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2020.4.5.1-py37_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::conda-4.8.3-py37_0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2020.4.5.1-py37_0, anaconda/osx-64::conda-4.8.3-py37_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.1.1-0, anaconda/osx-64::certifi-2020.4.5.1-py37_0, defaults/osx-64::conda-4.8.3-py37_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.4.5.1-py37_0, anaconda/osx-64::conda-4.8.3-py37_0, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::certifi-2020.4.5.1-py37_0, defaults/osx-64::ca-certificates-2020.1.1-0, defaults/osx-64::c

In [4]:
%conda install -c anaconda wget

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [21]:
%conda install -c anaconda biopython

Solving environment: done

## Package Plan ##

  environment location: /Users/smithded/opt/anaconda3

  added / updated specs:
    - biopython


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    biopython-1.76             |   py37h1de35cc_0         2.6 MB  anaconda
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

  biopython          anaconda/osx-64::biopython-1.76-py37h1de35cc_0



biopython-1.76       | 2.6 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


Assembly summary located at ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/assembly_summary.txt.

Elements of strategy for acquiring and parsing data and generating kmer frequences were influenced by code from www.github.com/smaelgol/PlasFlow - https://www.ncbi.nlm.nih.gov/pubmed/29346586 and www.gitlab.com/sirarredondo/analysis_mlplasmids - https://www.ncbi.nlm.nih.gov/pubmed/30383524.


In [27]:
!cat Data/assembly_summary.txt | grep "Bacteroides" | grep "Complete" > Data/Bacteroides.txt 


In [28]:
!cat Data/assembly_summary.txt | grep "Parabacteroides" | grep "Complete" > Data/Parabacteroides.txt 


In [29]:
!cat Data/Bacteroides.txt Data/Parabacteroides.txt > Data/Genomes.txt 

In [32]:
!cat Data/Genomes.txt | cut -f20 > Data/DownloadRefSeq.txt

In [43]:
!mkdir Data/Downloads

In [4]:
  #in shell  - for next in $(cat Data/DownloadRefSeq.txt); do wget -v -a log.txt -P Data/Downloads/ -nc "$next"/*[0-9]_genomic.fna.gz; done

In [185]:
# Manually downloaded Parabacteroides/Bacteroides plasmids not included in complete assemblies as per their respective NCBI Genomes pages (2020-05-04 DL date)
!ls Data/Downloads/Manualplasmids/


AP022661.1_plasmid.fna        NZ_ATFI01000025.1_plasmid.fna
AP022662.1_plasmid.fna        NZ_CM004507.1_plasmid.fna
AP022663.1_plasmid.fna        NZ_CM004522.1_plasmid.fna
AP022664.1_plasmid.fna        NZ_CM004523.1 _plasmid.fna
CP050955.1_plasmid.fna        NZ_CM018319.1_plasmid.fna
NZ_ATFI01000024.1_plasmid.fna


In [177]:
!gunzip Data/Downloads/*.gz

In [190]:
!for i in Data/Downloads/*.fna; do python Scripts/PlasmidorContig.py $i; done

In [191]:
!for i in Data/Downloads/Manualplasmids/*.fna; do python Scripts/PlasmidorContig.py $i; done
# moved all .fasta output files to /Data/Sequences

In [ ]:
### Will not execute via sh in Jupyter, to be run in local shell in appropriate folder and output to be moved to Split dirs below
#for i in *.fasta; do splitter -sequence $i -outseq Split500kb-$i -size 500000; done
cat Split* > Data/500kbcombined.fasta
#add in _ to for splitting first 2 chars, remove white space after
#AP022661.1 AP022662.1 AP022663.1 AP022664.1 CP050955.1

In [199]:
#!/usr/bin/python
from Bio import SeqIO
import pandas as pd
rows = []
fh = open("Data/500kbcombined.fasta", "r")
for seq_record in SeqIO.parse(fh, "fasta"):
    Desc = seq_record.description.split("_")
    Acc1 = str(Desc[0:1])[2:-2]
    Acc2 = str(Desc[1:2])[2:-2]
    Access = Acc1 + "_"+ Acc2
    Type = str(Desc[2:3])[2:-2]
    Seg = str(Desc[3:4])[2:-2]
    rows.append([Access,Type,Seg,str(seq_record.seq)])
df = pd.DataFrame(rows, columns=['Accession','Type','Range','Sequence'])

fh.close()

In [200]:
df.head()

,Accession,Type,Range,Sequence
0,AP_022661.1,plasmid,1-43945,TTTCATTAATCGACCCTTTTTCTGTACTTTTGATTTATAATAGTAC...
1,AP_022662.1,plasmid,1-8170,AGTTCATCAAGCAGCTTCTCCAGCTTCCTGAAAAGCTGCGGAGAGG...
2,AP_022663.1,plasmid,1-4316,ATGATGAAAACGAAACAACAGAAGAACAACAGTGTAACAGTCGTCA...
3,AP_022664.1,plasmid,1-2783,AGTTCATCAAAACCAAGTGATAACTGTCGTTTCTCTGCTTCTTCTT...
4,CP_050955.1,plasmid,1-2786,AGTATTATAGACAGCAAATGGACAATGCAAGGACAACCAGTAACCG...


In [206]:
#!/usr/bin/python
from Bio import SeqIO
import pandas as pd
rows = []
fh = open("Data/Sequences/Allseqs.fasta", "r")
for seq_record in SeqIO.parse(fh, "fasta"):
    Desc = seq_record.description.split("_")
    Acc1 = str(Desc[0:1])[2:-2]
    Acc2 = str(Desc[1:2])[2:-2]
    Access = Acc1 + "_"+ Acc2
    Type = str(Desc[2:3])[2:-2]
    rows.append([Access,Type,str(seq_record.seq)])
df = pd.DataFrame(rows, columns=['Accession','Type','Sequence'])

fh.close()

In [207]:
df.to_csv('Data/Rwork/Allseqs.csv', index=False)

In [201]:
df.to_csv('Data/Rwork/Split500kb.csv', index=False)

In [ ]:
"""R code: `Split500kb` <- read.csv("~/Documents/GitHub/ddnsmith/final-project/Data/Rwork/5000bpforR.csv")
Acc <- `Split500kb`[["Accession"]]
Type <- `Split500kb`[["Type"]]
seqs <- `Split500kb`[["Sequence"]]
library("BiocManager", lib.loc="~/Library/R/3.5/library")
library("Biostrings", lib.loc="~/Library/R/3.5/library")
sequences <- DNAStringSet(seqs)
kmer5 <- oligonucleotideFrequency(sequences, width=5,as.prob=TRUE)
kmer5df <-as.data.frame(kmer5)
dffinal <- data.frame(Acc,Type,kmer5df)
#code is the same for Allseqs and 9 mer output generated by changing width value to 9